# Idea
- Get first timestamp
- Resample Dataset to same timewindow
- Merge?

In [1]:
DEVICE_SOURCE = "Xi_sensor_2"

TIME_WINDOW = "30s"

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [4]:
sys_data_file_path = Path('/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Xi_sensor_2/merged_data.npz')
assert sys_data_file_path.exists(), 'Data not found'
vocab_file = Path("/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Xi_sensor_2/vocabulary.pkl")
assert vocab_file.exists(), f"Vocab file at {vocab_file} does not exist."

In [4]:
import sys
repo_base_path = Path("./../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

from py_dataset import get_all_files_df
from py_dataset import read_in_files
from py_dataset import feature_plotting
from py_dataset import feature_selection
from py_dataset import net_feature_engineering
from py_dataset import entropy_feature_engineering
from py_dataset import sys_func

from py_dataset.classes import DataSource

In [5]:
feature_path = repo_base_path / "data_merg" /"top_260_features.pkl"

with open(feature_path, "rb") as f:
    SELECTED_FEATURES = pickle.load(f)

In [6]:
data_path = Path('/media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/')
assert data_path.exists()

df = get_all_files_df.main(data_path)
df.head(1)

txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device1/3_thetick_4h/collections.txt
txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device2/1_normal/note.txt
                                           file_path
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...
RangeIndex(start=0, stop=622, step=1)
csv    530
zip     92
Name: filetype, dtype: int64
                                           file_path filetype  filesize_bytes
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...      csv           49419
data_source
SYS_data        92
FLS_data        91
KERN_data       91
RES_data        91
block_data      91
network_data    91
entropy_data    75
dtype: int64


,file_path,filetype,filesize_bytes,researcher_name,device_name,label,data_source,file_name,device
0,/media//DC/MAP_CreationOfNewDatasetsForDFL/...,csv,49419,Heqing,device1,1_normal,block_data,e4_5f_01_68_35_e6.csv,Heqing_device1


In [7]:
single_dev = df[df["device"] == DEVICE_SOURCE]
assert len(single_dev["device"].unique()) == 1
print(single_dev.shape)

df = single_dev

(57, 9)


### Get First TimeStamp

In [8]:
def prepare_files(row, resample_time_window):
    if row["filetype"] != "csv" and row["filetype"] != "zip":
        return None

    min_t, max_t, df_t = read_in_files.get_all_timestamps(row['file_path'], row["filetype"],
                                            resample_time_window=resample_time_window)

    row["min_t"] = min_t
    row["max_t"] = max_t

    return row

rows = []
for k, row in df.iterrows():
    x = prepare_files(row, resample_time_window=None)

    rows.append(x)

df_timestamps = pd.DataFrame(rows)

! Dropped rows in DataFrames: (1,)
timestamp   NaN
Name: 1264, dtype: float64
! Dropped rows in DataFrames: (1065,)
timestamp   NaN
Name: 1, dtype: float64
timestamp   NaN
Name: 3, dtype: float64
timestamp   NaN
Name: 5, dtype: float64
timestamp   NaN
Name: 7, dtype: float64
timestamp   NaN
Name: 9, dtype: float64
timestamp   NaN
Name: 11, dtype: float64
timestamp   NaN
Name: 13, dtype: float64
timestamp   NaN
Name: 15, dtype: float64
timestamp   NaN
Name: 17, dtype: float64
timestamp   NaN
Name: 19, dtype: float64
timestamp   NaN
Name: 21, dtype: float64
timestamp   NaN
Name: 23, dtype: float64
timestamp   NaN
Name: 25, dtype: float64
timestamp   NaN
Name: 27, dtype: float64
timestamp   NaN
Name: 29, dtype: float64
timestamp   NaN
Name: 31, dtype: float64
timestamp   NaN
Name: 33, dtype: float64
timestamp   NaN
Name: 35, dtype: float64
timestamp   NaN
Name: 37, dtype: float64
timestamp   NaN
Name: 39, dtype: float64
timestamp   NaN
Name: 41, dtype: float64
timestamp   NaN
Name: 43, dt

In [9]:
START = df_timestamps["min_t"].min()
f'Timestamps between: {df_timestamps["min_t"].min()} and {df_timestamps["max_t"].max()}'

'Timestamps between: 2023-11-04 07:38:05 and 2023-11-14 23:11:55'

### Resample files

In [10]:
def only_return_selected_columns(df: pd.DataFrame) -> pd.DataFrame:
    selected_cols = SELECTED_FEATURES & set(df.columns)
    return df[list(selected_cols)]

def create_window_sliced_depending_on_type(df: pd.DataFrame, start_timestamp: pd.Timestamp) -> pd.DataFrame:
    data_type = DataSource.from_columns(df.columns.tolist())
    print(f"Data type: {data_type}")

    match data_type:
        case (
            DataSource.FileSystem | DataSource.KernelEvents | DataSource.ResourceUsageEvents | DataSource.ResourceUsageEvents_RP4  | DataSource.BlockInputOutputEvents
        ):
            df = only_return_selected_columns(df)
            df = feature_plotting.test_transform_all_to_numeric_columns(df.copy())
            return df.resample(TIME_WINDOW, origin=start_timestamp).mean()
        case DataSource.NetRecords:
            mean_df = net_feature_engineering.create_mean_net_df_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            return only_return_selected_columns(mean_df)
        case DataSource.Entropy:
            if "entropy_file_count" not in SELECTED_FEATURES:
                return None
            return entropy_feature_engineering.resample_high_entropy_grouping_by_label_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            
        case DataSource.SystemCalls:
            raise NotImplementedError("SystemCalls not implemented yet")
        

def check_if_keep_source(columns):
    data_type = DataSource.from_columns(columns.tolist())
    if data_type in [DataSource.SystemCalls, DataSource.Entropy, DataSource.NetRecords]:
        return True

    chosen_cols_from_source = data_type.value & SELECTED_FEATURES

    return any(chosen_cols_from_source)

In [11]:

df_mean_collection: list[pd.DataFrame] = []

for source, group_df in df.groupby(["data_source"]):
    dfs_same_source = []

    checked_source = False
    for k, row in group_df.iterrows():
        if row["filetype"] != "csv":
            print(f"Skipping {row['file_path']}")
            continue


        df_t = read_in_files.get_all_columns(row['file_path'], row["filetype"])
        if not checked_source:
            if not check_if_keep_source(df_t.columns):
                print(f"Skipping {source}")
                break
            checked_source = True

        try:
            df_mean = create_window_sliced_depending_on_type(df_t, start_timestamp=START)
        except Exception as ex:
            print(f"Problem in file {row['file_path']}")
            raise ex
        
        if df_mean is None:
            print(f"Skipping {row['file_path']}")
            continue
        
        df_mean = df_mean.copy()
        print(f"Shape: {df_mean.shape}")
        df_mean["label"] = row["label"]
        df_mean["device"] = row["device"]
        df_mean["data_source"] = row["data_source"]

        # print(f"Start and End of Index Timestamp: {df_mean.index[0]} - {df_mean.index[-1]}")
        dfs_same_source.append(df_mean)

    if len(dfs_same_source) == 0:
        print(f"Skipping {source}")
        continue
    same_df = pd.concat(dfs_same_source, axis=0)
    print(f"same_df shape: {same_df.shape}")
    df_mean_collection.append(same_df)
    # break

len(df_mean_collection)

/tmp/ipykernel_44461/1828661645.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for source, group_df in df.groupby(["data_source"]):


Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (541, 54)
Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (482, 54)
Data type: DataSource.FileSystem
Shape: (483, 54)
same_df shape: (4390, 57)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (483, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (541, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (482, 56)
Data type: DataSource.KernelEvents
Shape: (483, 56)
Data type: DataSource.KernelEvents
Shape: (484, 56)
same_df shape: (4397, 59)
Data type: DataSource.Resource

5

#### Remove Dublicate Columns and Concatenate

=> some dont work => if Index error use merge manually

In [13]:
try:
    pd.concat(df_mean_collection, axis=1)
except Exception as ex:
    print(ex)

In [14]:
from collections import defaultdict

cols = defaultdict(int)
for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        cols[k] +=1

print("Dublicates:", [k for k, v in cols.items() if v > 1])

to_ignore = ["data_source", "label", "device"]

infos = pd.DataFrame(columns=to_ignore)

for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        if k in to_ignore:
            continue

        if cols[k] > 1:
            print(k)

            dfs.rename(columns={k: f'{k}_{dfs["data_source"].iloc[0]}'}, inplace=True)

    assert dfs["device"].nunique() == 1
    assert dfs["label"].nunique() == 9
    assert dfs["data_source"].nunique() == 1
    
    infos = pd.concat([infos,dfs[to_ignore]], axis=0)

    # for t,row in dfs.iterrows():
        # infos.append({"device": row["device"], "label": row["label"], "data_source": row["data_source"], "timestamp": t})
    
    dfs.drop(columns=to_ignore, inplace=True)

Dublicates: ['ext4:ext4_da_update_reserve_space', 'ext4:ext4_writepages_result', 'ext4:ext4_journal_start', 'writeback:sb_clear_inode_writeback', 'jbd2:jbd2_handle_start', 'writeback:writeback_dirty_inode_enqueue', 'ext4:ext4_es_lookup_extent_enter', 'block:block_bio_backmerge', 'block:block_touch_buffer', 'writeback:writeback_mark_inode_dirty', 'writeback:writeback_dirty_inode', 'ext4:ext4_unlink_enter', 'block:block_bio_remap', 'jbd2:jbd2_start_commit', 'jbd2:jbd2_checkpoint_stats', 'ext4:ext4_releasepage', 'jbd2:jbd2_handle_stats', 'writeback:global_dirty_state', 'ext4:ext4_free_inode', 'writeback:writeback_dirty_page', 'block:block_unplug', 'seconds', 'filemap:mm_filemap_delete_from_page_cache', 'filemap:mm_filemap_add_to_page_cache', 'block:block_getrq', 'block:block_dirty_buffer', 'label', 'device', 'data_source', 'tcp:tcp_probe', 'kmem:mm_page_alloc_zone_locked', 'sched:sched_process_exec', 'fib:fib_table_lookup', 'kmem:kmem_cache_alloc', 'gpio:gpio_value', 'kmem:mm_page_free', 

In [15]:
df_all = pd.concat(df_mean_collection[:-1], axis=1).merge(df_mean_collection[-1], how='outer', left_index=True, right_index=True)
df_all.sample(2)

,ext4:ext4_ext_rm_leaf,ext4:ext4_da_update_reserve_space_FLS_data,block:block_rq_issue,ext4:ext4_writepages_result_FLS_data,ext4:ext4_readpage,ext4:ext4_journal_start_FLS_data,ext4:ext4_da_reserve_space,ext4:ext4_es_lookup_extent_exit,writeback:sb_clear_inode_writeback_FLS_data,jbd2:jbd2_handle_start_FLS_data,...,filemap:mm_filemap_delete_from_page_cache_RES_data,LLC-loads,filemap:mm_filemap_add_to_page_cache_RES_data,iobusytime,block:block_dirty_buffer_RES_data,util,write_ops,write_kbs,(OLD) VarianceLength,(OLD) AverageLength
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 16:28:05,26.0,26.0,28.0,26.0,0.0,1014.0,26.0,1258.0,26.0,869.0,...,2790.666667,3.308076e+08,17.666667,360.0,37.0,8.353333,23.6,3.066667,471563.617183,957.818182
2023-11-06 16:32:35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,375.0,0.0,0.0,...,855.666667,2.832193e+08,0.000000,0.0,0.0,0.000000,0.0,0.000000,418667.416528,1068.560322


In [16]:
df_all = df_all.merge(infos["label"], how='outer', left_index=True, right_index=True)

In [17]:
df_all.shape, len(SELECTED_FEATURES)

((21974, 189), 260)

In [18]:
df_all

,ext4:ext4_ext_rm_leaf,ext4:ext4_da_update_reserve_space_FLS_data,block:block_rq_issue,ext4:ext4_writepages_result_FLS_data,ext4:ext4_readpage,ext4:ext4_journal_start_FLS_data,ext4:ext4_da_reserve_space,ext4:ext4_es_lookup_extent_exit,writeback:sb_clear_inode_writeback_FLS_data,jbd2:jbd2_handle_start_FLS_data,...,LLC-loads,filemap:mm_filemap_add_to_page_cache_RES_data,iobusytime,block:block_dirty_buffer_RES_data,util,write_ops,write_kbs,(OLD) VarianceLength,(OLD) AverageLength,label
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,223631535.0,1.0,0.5,3.0,5.333333,1.066667,0.2,399283.940058,1096.969144,1_normal
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,223631535.0,1.0,0.5,3.0,5.333333,1.066667,0.2,399283.940058,1096.969144,1_normal
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,223631535.0,1.0,0.5,3.0,5.333333,1.066667,0.2,399283.940058,1096.969144,1_normal
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,223631535.0,1.0,0.5,3.0,5.333333,1.066667,0.2,399283.940058,1096.969144,1_normal
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,223631535.0,1.0,0.5,3.0,5.333333,1.066667,0.2,399283.940058,1096.969144,1_normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-14 23:11:05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464.0,0.0,0.0,...,236338858.5,0.0,0.0,0.0,0.000000,0.000000,0.0,367115.335839,1155.202960,4_bashlite
2023-11-14 23:11:05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464.0,0.0,0.0,...,236338858.5,0.0,0.0,0.0,0.000000,0.000000,0.0,367115.335839,1155.202960,4_bashlite
2023-11-14 23:11:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,238396249.0,0.0,0.0,0.0,0.000000,0.000000,0.0,477506.786277,652.419643,4_bashlite
2023-11-14 23:11:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,238396249.0,0.0,0.0,0.0,0.000000,0.000000,0.0,477506.786277,652.419643,4_bashlite


# Add Sys Call Data

In [5]:
def read_in_preprocessed_Xi_sensor_2():
    with np.load(file_path, allow_pickle=True) as npz:
        print(npz.files)
        X = npz['arr_0']
    print(X.shape)

    vectorizer = sys_func.create_CountVectorizer(vocab_file=vocab_file)
    cols = vectorizer.get_feature_names_out()
    cols = np.append(cols, [['timestamp'], ['label']])
    print (cols.shape)
    df = pd.DataFrame(X, columns=cols)
    # df.set_index('timestamp', inplace=True)
    read_in_files._clean_and_set_index_timestamp(df)

    label_encoder = sys_func.create_label_encoder_for_labels()
    df['label'] = label_encoder.inverse_transform(df['label'].values.astype(int))

    vectors = df.copy()
    print(vectors.shape)
    
    return vectors

sys_df = read_in_preprocessed_Xi_sensor_2()
# sys_df = only_return_selected_columns(sys_df)
# sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

['arr_0']
(10136, 156)


NameError: name 'sys_func' is not defined

In [20]:
sys_df.head(2)

,accept,accept4,access,adjtimex,bind,brk,capget,capset,chdir,chmod,...,uname,unlink,unlinkat,utimensat,utimes,wait4,waitid,write,writev,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-04 07:38:05,0.0,0.000067,0.007107,0.001046,0.000089,0.009085,0.000485,0.000167,0.000088,0.0,...,0.001817,0.000025,0.0,0.0,0.0,0.003129,0.000110,0.005613,0.000060,1_normal
2023-11-04 07:38:16,0.0,0.000557,0.014561,0.002602,0.000000,0.017323,0.000804,0.000208,0.000145,0.0,...,0.004117,0.000124,0.0,0.0,0.0,0.006929,0.000273,0.013758,0.000149,1_normal


In [28]:
if sys_df["label"].unique().shape[0] != 9:
    raise Exception(f'Not all labels!!!: {sys_df["label"].unique()}')
sys_df["label"].unique()

array(['1_normal', '9_xmrig', '3_thetick', '5_httpbackdoor', '6_beurk',
       '7_backdoor', '2_ransomware', '8_bdvl', '4_bashlite'], dtype=object)

In [29]:
sys_df = only_return_selected_columns(sys_df)
sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

In [30]:
sys_df_mean.head(2)

,rename,getsockopt,capget,connect,llseek,pipe2,close,ftruncate64,sendmsg,sigreturn,...,getpgrp,chdir,dup2,getuid32,setgroups32,capset,dup,setsockopt,exit,open
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-04 07:38:05,0.000093,0.003197,0.000655,0.011853,0.276449,0.000591,0.298918,0.000521,0.019471,0.002557,...,0.0,0.000101,0.003937,0.006448,0.0,0.000193,0.000313,0.013174,0.000000,0.0
2023-11-04 07:38:35,0.000083,0.004019,0.000690,0.014134,0.375092,0.000619,0.323217,0.000647,0.022788,0.002886,...,0.0,0.000082,0.004358,0.007245,0.0,0.000211,0.000301,0.015892,0.000053,0.0


In [31]:
all_df1 = df_all.merge(sys_df_mean, how='outer', left_index=True, right_index=True)
print(f"all_df1 shape: {all_df1.shape}")

all_df1 shape: (48241, 289)


In [32]:
all_df1.head(4)

,ext4:ext4_ext_rm_leaf,ext4:ext4_da_update_reserve_space_FLS_data,block:block_rq_issue,ext4:ext4_writepages_result_FLS_data,ext4:ext4_readpage,ext4:ext4_journal_start_FLS_data,ext4:ext4_da_reserve_space,ext4:ext4_es_lookup_extent_exit,writeback:sb_clear_inode_writeback_FLS_data,jbd2:jbd2_handle_start_FLS_data,...,getpgrp,chdir,dup2,getuid32,setgroups32,capset,dup,setsockopt,exit,open
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,0.0,0.000101,0.003937,0.006448,0.0,0.000193,0.000313,0.013174,0.0,0.0
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,0.0,0.000101,0.003937,0.006448,0.0,0.000193,0.000313,0.013174,0.0,0.0
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,0.0,0.000101,0.003937,0.006448,0.0,0.000193,0.000313,0.013174,0.0,0.0
2023-11-04 07:38:05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,608.0,0.0,0.0,...,0.0,0.000101,0.003937,0.006448,0.0,0.000193,0.000313,0.013174,0.0,0.0


In [33]:
spath = repo_base_path / "training" / "data" / f"all_df_{TIME_WINDOW}_{DEVICE_SOURCE}.csv"

all_df1.to_csv(str(spath))